# Notebook to save predictions to 51

## Imports

In [2]:
import fiftyone as fo
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
import cv2
from PIL import Image # TODO to change 
from torchvision import transforms
from torchvision.transforms import (
    ToTensor,
    ToPILImage,
    Resize
)

## Get the dataset and filter it

In [3]:
dataset = fo.load_dataset("esmart_wip")

In [4]:
train_dataset_51 = dataset.match_tags("TRAIN_ROAD_COND")
val_dataset_51 = dataset.match_tags("VAL_ROAD_COND")

In [5]:
fo.launch_app(dataset)

Dataset:          esmart_wip
Media type:       image
Num samples:      6380
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

## Run Detect.py on the validation set

In [ ]:
 !python detect.py 
    --weights runs/train/exp385/weights/best.pt 
    --source data/multitasks.yaml 
    --img-size 640 640 
    --nosave 
    --save-txt 
    --fiftyone


## Delete old predictions for road_condtions in order to save the new ones

In [27]:
for sample in val_dataset_51:
#     print(sample)
    try:
        del sample["classif_road_cond_yolo"]
        del sample["classif_road_cond_yolo_2"]
        sample.save()
    except:
        a = 1

## Save predictions to Fiftyone

In [28]:
def get_prediction(path):
    with open(path, 'r') as f:
        lines = f.readlines()
        condition, confidence = lines[0].split(',')[0], lines[0].split(',')[1]
    return condition, confidence

In [29]:
directory = '/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls'
with fo.ProgressBar() as pb:
    for sample in pb(val_dataset_51):
        try:
            img = sample.filepath.split('/')[-1]
            if img.endswith('.jpg'):
                img_name = img.replace('jpg','txt')
            elif img.endswith('.png'):
                img_name = img.replace('png','txt')
            path = f"{directory}/{img_name}"
            pred, conf = get_prediction(path)
            sample["classif_road_cond_yolo"] = fo.Classifications(classifications=[fo.Classification(label=pred, confidence=conf)])
            sample.save()
        except:
            print(img_name)